In [2]:
import sys
sys.path.append("../..")
from sources import *
import os
from IPython.display import clear_output
from pymongo import MongoClient
import db
client = MongoClient('localhost', 27017)
db = client["stats"]
import time

In [3]:
from sources import worldbank as wb
wb.indicators.load_all()

In [21]:
indicators_to_retrieve = ["SP.POP.TOTL", "SH.DYN.MORT", "EN.ATM.CO2.EPC", "NY.GNP.PCAP.CD", "EG.ELC.ACCS.ZS"]

In [33]:
from geo.data import world
def process_wb_data(data):
    new_data =[];
    for dp in data:
        if dp['entity'] in world:
            new_data.append({'entity':dp['entity'].lower(), 'date': dp['date'], 'value': dp['value']})
        else:
            pass
        
    return new_data

In [35]:
i = 0
j = 0
years= list(map(str, list(range(1962,2018))))
for ind in wb.indicators.Indicators:
    if ind.name in indicators_to_retrieve:        
        try:
            data = wb.indicators.get(name=ind.name, years=years, countries=world)
            ddata = process_wb_data(data['data'])
            print (data['name'])
            print (data['description'])            
            time.sleep(5)        
            db["indicators"].update_one({ "name" : data['name']}, 
                           {"$set":{"name" : data['name'], 
                                    "description" : data['description'], 
                                    "years" : years, 
                                    "sourceNote" : ind.sourcenote,
                                    "connector" : data['source'],
                                    "datasource" : ind.datasource,
                                    "sourceurl" : data['sourceurl'], 
                                    "keyField" : "entity",
                                    "pivotField" : "date",
                                    "valueField" : "value",
                                    'data' : ddata}}, True)
        except ValueError:
            print(ValueError)
            pass

EGELCACCSZS
Access to electricity (% of population)
NYGNPPCAPCD
GNI per capita, Atlas method (current US$)
SHDYNMORT
Mortality rate, under-5 (per 1,000 live births)
SPPOPTOTL
Population, total


In [ ]:
import urllib
import json
url = 'http://api.worldbank.org/indicators?format=json&per_page=' + \
        str(20000)

response = urllib.request.urlopen(url)
data = response.read().decode('utf-8')
data = json.loads(data)


In [ ]:
print(data[1][0])